In [1]:
# whether to log each feature and sequence status
verbose = True

In [2]:
import gc
import os
import numpy as np
import pandas as pd
import json
import datetime
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.utils import shuffle
import sys
from shutil import rmtree

from keras.utils import to_categorical
sys.path.append('..')

2023-04-02 23:44:41.529653: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-02 23:44:41.689892: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /app/lib
2023-04-02 23:44:41.689910: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-02 23:44:41.719374: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-02 23:44:42.276326: W tensorflow/

In [3]:
__file__ = os.path.abspath('helper_precomputing_CNN_features.ipynb')
DNN_lib_path = Path(__file__).parents[1].__str__()
path_data = DNN_lib_path + '/data_cnn_ts_3d/'
path_cache = DNN_lib_path + '/cache/'

custom_model_name = 'ResNet50_test'
path_features = path_cache + '/features/' + custom_model_name


In [4]:
# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler(f"{DNN_lib_path}/logs_{custom_model_name}_training.log"),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()
# make logger aware of any uncaught exceptions
def handle_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return

    logger.error("Uncaught exception", exc_info=(exc_type, exc_value, exc_traceback))
sys.excepthook = handle_exception

In [5]:
from deepvideoclassification.architectures import Architecture
from deepvideoclassification.data import Data

# Create Data Object

In [6]:
label_df = pd.read_csv(path_data + 'labels.csv', usecols=['video','frame','label','split']).sort_values(['video', 'frame'])
label_df

,video,frame,label,split
0,trajs_2017-03-09_Ut_3062_door_3,trajs_2017-03-09_Ut_3062_door_3-0000.npy,pre-deboarding,test
1,trajs_2017-03-09_Ut_3062_door_3,trajs_2017-03-09_Ut_3062_door_3-0001.npy,pre-deboarding,test
2,trajs_2017-03-09_Ut_3062_door_3,trajs_2017-03-09_Ut_3062_door_3-0002.npy,pre-deboarding,test
3,trajs_2017-03-09_Ut_3062_door_3,trajs_2017-03-09_Ut_3062_door_3-0003.npy,pre-deboarding,test
4,trajs_2017-03-09_Ut_3062_door_3,trajs_2017-03-09_Ut_3062_door_3-0004.npy,pre-deboarding,test
...,...,...,...,...
5995,trajs_2018-05-30_Ut_3058_door_3,trajs_2018-05-30_Ut_3058_door_3-0195.npy,post-boarding,valid
5996,trajs_2018-05-30_Ut_3058_door_3,trajs_2018-05-30_Ut_3058_door_3-0196.npy,post-boarding,valid
5997,trajs_2018-05-30_Ut_3058_door_3,trajs_2018-05-30_Ut_3058_door_3-0197.npy,post-boarding,valid
5998,trajs_2018-05-30_Ut_3058_door_3,trajs_2018-05-30_Ut_3058_door_3-0198.npy,post-boarding,valid


In [7]:
labels_dummied = pd.get_dummies(label_df, columns=['label'])
labels_dummied 


,video,frame,split,label_boarding,label_deboarding,label_phase-change,label_post-boarding,label_pre-deboarding
0,trajs_2017-03-09_Ut_3062_door_3,trajs_2017-03-09_Ut_3062_door_3-0000.npy,test,0,0,0,0,1
1,trajs_2017-03-09_Ut_3062_door_3,trajs_2017-03-09_Ut_3062_door_3-0001.npy,test,0,0,0,0,1
2,trajs_2017-03-09_Ut_3062_door_3,trajs_2017-03-09_Ut_3062_door_3-0002.npy,test,0,0,0,0,1
3,trajs_2017-03-09_Ut_3062_door_3,trajs_2017-03-09_Ut_3062_door_3-0003.npy,test,0,0,0,0,1
4,trajs_2017-03-09_Ut_3062_door_3,trajs_2017-03-09_Ut_3062_door_3-0004.npy,test,0,0,0,0,1
...,...,...,...,...,...,...,...,...
5995,trajs_2018-05-30_Ut_3058_door_3,trajs_2018-05-30_Ut_3058_door_3-0195.npy,valid,0,0,0,1,0
5996,trajs_2018-05-30_Ut_3058_door_3,trajs_2018-05-30_Ut_3058_door_3-0196.npy,valid,0,0,0,1,0
5997,trajs_2018-05-30_Ut_3058_door_3,trajs_2018-05-30_Ut_3058_door_3-0197.npy,valid,0,0,0,1,0
5998,trajs_2018-05-30_Ut_3058_door_3,trajs_2018-05-30_Ut_3058_door_3-0198.npy,valid,0,0,0,1,0


In [8]:
numerization_dict = dict({
    None : 4,
    np.nan: 4, 
    'pre-deboarding': 0,
    'deboarding': 1,
    'phase-change': 2,
    'boarding': 3,
    'post-boarding': 0
})

class Data(object):

    def __init__(self, sequence_length,
                 return_CNN_features=True,
                 pretrained_model_name=None,
                 pooling=None,
                 frame_size=None,
                 custom_model_name=None,
                 _bed=False,
                 verbose=True,
                 return_generator = False):
        """
        Data object constructor
        
        
        :sequence_length: number of frames in sequence to be returned by Data object
        :return_CNN_features: whether to return precomputed features or return frames (or sequences of features/frames if sequence_length>1)

        :return_features: if True then return features (or sequences of feature) from pretrained model, if False then return frames (or sequences of frames)        
        :pretrained_model_name: name of pretrained model (or None if not using pretrained model e.g. for 3D-CNN)
        :pooling: name of pooling variant (or None if not using pretrained model e.g. for 3D-CNN)
        :frame_size: size that frames are resized to (this is looked up for pretrained models)
        :aug3mentation: whether to apply data augmentation (horizontal flips)
        :oversampling: whether to apply oversampling to create class balance
        
        :model_weights_path: path to custom model weights if we want to load CNN model we've fine-tuned to produce features (e.g. for LRCNN)
        :custom_model_name: custom output name to append to pretrained model name
        
        :return_generator: if True and sequence_length > 1 and return_CNN_features == False, then do not return dataset, instead construct h5 file with sequences for each split and return generator that samples from that (dataset of sequecne frames too big to load into memory)
        :batch_size: size of batches that generator must return
        
        :verbose: whether to log details
        
        Notes: 
        * if pretrained_model_name != None and return_CNN_features=False then will first apply preprocessor to frames (or frame sequences)
        * if return_generator = True and sequence_length > 1 and return_CNN_features == False, large h5 files will be created in cache before returning generator
        """

        # required params
        self.sequence_length = sequence_length
        self.frame_size = frame_size

        # optional params
        self.pretrained_model_name = pretrained_model_name
        self.pooling = pooling
        self.return_CNN_features = return_CNN_features
        self.custom_model_name = custom_model_name
        self.return_generator = return_generator

        self.bed = _bed
        self.frame_size = frame_size

        self.verbose = verbose

        self.x_train = []
        self.y_train = []
        #
        self.x_valid = []
        self.y_valid = []
        #
        self.x_test = []
        self.y_test = []

        ################
        ### Prepare data
        ################

        #Label loading
        self.labels = pd.read_csv(path_data + 'labels.csv', usecols=['video','frame','label','split']).sort_values(['video', 'frame'])
        
        # get label columns list and build label map dict
        label_columns = []
        label_map = {}
        label_map_idx = 0
        for i, col in enumerate(labels_dummied.columns):
            if col[:6] == 'label_':
                label_columns.append(col)
                label_map[label_map_idx] = col
                label_map_idx+=1

        self.label_map = label_map

        # get video paths
        self.path_videos = [f"{path_data}/{video}" for video in os.listdir(path_data) if os.path.isdir(f"{path_data}/{video}")]

        # check that there is 1 frame file for each label file and raise error if they don't match
        path_frames = []
        for folder, subs, files in os.walk(path_data):
            for filename in files:
                if self.bed & (filename[-4:].lower() == '.npy'):
                    path_frames.append(os.path.abspath(
                        os.path.join(folder, filename)))

                if filename[-4:].lower() == '.jpg' or filename[-4:].lower() == 'jpeg' or filename[-4:].lower() == '.png':
                    path_frames.append(os.path.abspath(
                        os.path.join(folder, filename)))

        if len(path_frames) != len(self.labels):
            error_msg = 'IMPORTANT ERROR: Number of frames ({}) in /data/ video folders needs to match number of labels ({}) in labels.csv - use notebooks/helper_check_frames_against_labels.ipynb to investigate... Note, only labels.csv and the frames you want to use (in video subfolders) should be in /data/'.format(
                len(path_frames), len(self.labels))
            logger.info(error_msg)
            raise ValueError(error_msg)

        # pull number of classes from labels shape
        self.num_classes = self.labels['label'].nunique()

        # create dict mapping video to train/valid/test split assignment
        video_splits = self.labels[['video', 'split']].drop_duplicates()
        video_splits.set_index("video", inplace=True)
        video_splits = video_splits.to_dict()['split']
        self.video_splits = video_splits

        # precompute resized frames (won't recompute if already resized)
        #resize_frames(self.frame_size, _bed = self.bed, _verbose = self.verbose)

        ###################################
        ### load features / build sequences
        ###################################

        # load features/frames from all videos and concat into big array for each of train, valid and test
        assert self.return_CNN_features

        if verbose:
            logging.info(
                "Loading features sequence data into memory [may take a few minutes]")

        #####################
        ### feature sequences
        #####################

        path_features = path_cache + 'features/' + self.custom_model_name
        path_labels = path_cache + 'labels/'
        
        bes_names = [be for be in os.listdir(path_data) if os.path.isdir(f"{path_data}/{be}")]

        # loop over all vids and load precomputed features into memory as sequences
        for c, be_name in enumerate(bes_names):

            path_be = f'{path_data}/{be_name}'

            if verbose:
                logging.info("Loading features sequence data into memory {}/{}".format(c+1,len(path_be)))

            ### create sequence: features
            # load precomputed features
            features = np.load(f"{path_features}/{be_name}.npy")
            # build sequences
            x = []
            for i in range(self.sequence_length, len(features) + 1):
                x.append(features[i-self.sequence_length:i])
            x = np.array(x)
            

            # temp lists to store sequences
            be_labels = self.labels[self.labels.video == be_name]
            y = []
            for i in range(self.sequence_length, len(be_labels) + 1):
                label = be_labels.label.iloc[i-1]
                if (label is None) or (label == np.nan):
                    label = 'nan'
                y.append(label)
            y = np.array(list(map(numerization_dict.get, y)))
            y = to_categorical(y, num_classes=5)

            assert len(x) == len(y), f'Length of features ({len(x)}) does not match length of labels ({len(y)})'

            ### build output
            if self.video_splits[be_name] == "train":
                self.x_train.append(x)
                self.y_train.append(y)
            if self.video_splits[be_name] == "valid":
                self.x_valid.append(x)
                self.y_valid.append(y)
            if self.video_splits[be_name] == "test":
                self.x_test.append(x)
                self.y_test.append(y)

        #################################
        ### get file paths for each split
        #################################
        #
        # Note: only makes sense for sequence_length = 1

        # get file paths: train
        dflab = self.labels[self.labels['split'] == 'train']
        self.paths_train = list(
            path_data + dflab['video'] + "/" + dflab['frame'])

        # get file paths: valid
        dflab = self.labels[self.labels['split'] == 'valid']
        self.paths_valid = list(
            path_data + dflab['video'] + "/" + dflab['frame'])

        # get file paths: test
        dflab = self.labels[self.labels['split'] == 'test']
        self.paths_test = list(
            path_data + dflab['video'] + "/" + dflab['frame'])

        #################################################
        ### reshape list outputs (if not using generator)
        #################################################

        ## e.g. (9846, 224, 224, 3) for frames [return_CNN_features=True]
        ## or  (9846, 512) for features [return_CNN_features=False]
        self.x_train = np.concatenate(self.x_train, axis=0)
        self.y_train = np.concatenate(self.y_train, axis=0)
        self.x_valid = np.concatenate(self.x_valid, axis=0)
        self.y_valid = np.concatenate(self.y_valid, axis=0)
        self.x_test = np.concatenate(self.x_test, axis=0)
        self.y_test = np.concatenate(self.y_test, axis=0)

        self.total_rows_train = len(self.x_train)
        self.total_rows_valid = len(self.x_valid)
        self.total_rows_test = len(self.x_test)

        # shuffle train and validation set
        self.x_train, self.y_train = shuffle(self.x_train, self.y_train)
        self.x_valid, self.y_valid = shuffle(self.x_valid, self.y_valid)

        # update progress
        if self.verbose:
            print("Done initializing data with #samples: train={}, valid={}, test={}".format(
                self.total_rows_train, self.total_rows_valid, self.total_rows_test))


In [19]:
experiment = {
    'architecture': 'video_mlp_concat',
    'dropout': 0.2,
    'layer_1_size': 256,
    'layer_2_size': 512,
    'layer_3_size': 256,
    'model_id': 1,
    'pooling': 'max',
    'pretrained_model_name': 'SimpleRNN',
    'custom_model_name': custom_model_name,
    'path_features': f'/cache/{custom_model_name}',
    'sequence_length': 20,
    'sequence_model': "LSTM",
    'sequence_model_layers': 3,
    'frame_size': (32, 32)
}


In [20]:
be_data = Data(
    sequence_length = experiment['sequence_length'],
    return_CNN_features = True, 
    pretrained_model_name = experiment['pretrained_model_name'],
    pooling = experiment['pooling'],
    frame_size = experiment['frame_size'],
    custom_model_name= experiment['custom_model_name'],
    return_generator = False,
    _bed = True
)

2023-04-02 23:51:51,801 [MainThread  ] [INFO ]  Loading features sequence data into memory [may take a few minutes]
2023-04-02 23:51:51,803 [MainThread  ] [INFO ]  Loading features sequence data into memory 1/139
2023-04-02 23:51:51,814 [MainThread  ] [INFO ]  Loading features sequence data into memory 2/139
2023-04-02 23:51:51,825 [MainThread  ] [INFO ]  Loading features sequence data into memory 3/139
2023-04-02 23:51:51,836 [MainThread  ] [INFO ]  Loading features sequence data into memory 4/139
2023-04-02 23:51:51,846 [MainThread  ] [INFO ]  Loading features sequence data into memory 5/139
2023-04-02 23:51:51,856 [MainThread  ] [INFO ]  Loading features sequence data into memory 6/139
2023-04-02 23:51:51,883 [MainThread  ] [INFO ]  Loading features sequence data into memory 7/139
2023-04-02 23:51:51,947 [MainThread  ] [INFO ]  Loading features sequence data into memory 8/139
2023-04-02 23:51:52,012 [MainThread  ] [INFO ]  Loading features sequence data into memory 9/139
2023-04-02 

Done initializing data with #samples: train=362, valid=724, test=4344


In [21]:
# delete existing results
if os.path.exists(DNN_lib_path + '/models/' + str(experiment["model_id"]) + '/results.json'):
    rmtree(DNN_lib_path + '/models/' + str(experiment["model_id"]) + '/')
# create models folder if doesn't exist
if not os.path.exists(DNN_lib_path + '/models/'):
    os.makedirs(DNN_lib_path + '/models/')


In [22]:
print(str(experiment["model_id"]) + "   " + "X"*60)
print(experiment)

architecture = Architecture(model_id = experiment['model_id'], 
                            architecture = experiment['architecture'], 
                            sequence_length = experiment['sequence_length'], 
                            pretrained_model_name = experiment['pretrained_model_name'],
                            custom_model_name = experiment['custom_model_name'],
                            pooling = experiment['pooling'],
                            sequence_model = experiment['sequence_model'],
                            sequence_model_layers = experiment['sequence_model_layers'],
                            layer_1_size = experiment['layer_1_size'],
                            layer_2_size = experiment['layer_2_size'],
                            layer_3_size = experiment['layer_3_size'],
                            dropout = experiment['dropout'],
                            _bed = True,
                            verbose=True,
                            data = be_data)

1   XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
{'architecture': 'video_mlp_concat', 'dropout': 0.2, 'layer_1_size': 256, 'layer_2_size': 512, 'layer_3_size': 256, 'model_id': 1, 'pooling': 'max', 'pretrained_model_name': 'SimpleRNN', 'custom_model_name': 'ResNet50_test', 'path_features': '/cache/ResNet50_test', 'sequence_length': 20, 'sequence_model': 'LSTM', 'sequence_model_layers': 3, 'frame_size': (32, 32)}


KeyError: 'SimpleRNN'

In [ ]:
architecture.train_model()


Epoch 1: val_accuracy improved from -inf to 0.72514, saving model to /home/tiesbarendse/Documents/00_Uni/22_23/pap/Deep-Neural-Networks-for-Video-Classification/notebooks/../models/1/model_round_1.h5

Epoch 2: val_accuracy did not improve from 0.72514

Epoch 3: val_accuracy did not improve from 0.72514

Epoch 4: val_accuracy improved from 0.72514 to 0.73481, saving model to /home/tiesbarendse/Documents/00_Uni/22_23/pap/Deep-Neural-Networks-for-Video-Classification/notebooks/../models/1/model_round_1.h5

Epoch 5: val_accuracy did not improve from 0.73481

Epoch 6: val_accuracy improved from 0.73481 to 0.73619, saving model to /home/tiesbarendse/Documents/00_Uni/22_23/pap/Deep-Neural-Networks-for-Video-Classification/notebooks/../models/1/model_round_1.h5

Epoch 7: val_accuracy did not improve from 0.73619

Epoch 8: val_accuracy improved from 0.73619 to 0.73757, saving model to /home/tiesbarendse/Documents/00_Uni/22_23/pap/Deep-Neural-Networks-for-Video-Classification/notebooks/../model

2023-04-02 23:50:22.871188: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 711720960 exceeds 10% of free system memory.


136/136 [==============================] - 1s 9ms/step


2023-04-02 23:50:25,278 [MainThread  ] [INFO ]  {
    "architecture": "video_mlp_concat",
    "batch_size": 32,
    "bed": true,
    "convolution_kernel_size": 3,
    "custom_model_name": "ResNet50_test",
    "data_total_rows_test": 4344,
    "data_total_rows_train": 362,
    "data_total_rows_valid": 724,
    "dropout": 0.2,
    "fit_best_round": 3,
    "fit_dt_test_duration_seconds": "1",
    "fit_dt_test_end": "2023-04-02 23:50:24",
    "fit_dt_test_start": "2023-04-02 23:50:22",
    "fit_dt_train_duration_seconds": "27",
    "fit_dt_train_end": "2023-04-02 23:50:22",
    "fit_dt_train_start": "2023-04-02 23:49:55",
    "fit_num_epochs": 20,
    "fit_stopped_epoch1": 8,
    "fit_stopped_epoch2": 6,
    "fit_stopped_epoch3": 3,
    "fit_test_acc": 0.7651933701657458,
    "fit_train_acc": 0.9861878156661987,
    "fit_train_loss": 0.015030477195978165,
    "fit_val_acc": 0.738950252532959,
    "fit_val_loss": 1.9370166063308716,
    "frame_size": [
        224,
        224
    ],
    "l

{'model_id': 1, 'architecture': 'video_mlp_concat', 'sequence_length': 20, 'frame_size': (224, 224), 'pretrained_model_name': 'resnet50', 'custom_model_name': 'ResNet50_test', 'pooling': 'max', 'sequence_model': 'LSTM', 'sequence_model_layers': 2, 'layer_1_size': 256, 'layer_2_size': 512, 'layer_3_size': 256, 'dropout': 0.2, 'convolution_kernel_size': 3, 'model_weights_path': None, 'batch_size': 32, 'bed': True, 'verbose': True, 'num_features': 2048, 'path_model': '/home/tiesbarendse/Documents/00_Uni/22_23/pap/Deep-Neural-Networks-for-Video-Classification/notebooks/../models/1/', 'data_total_rows_train': 362, 'data_total_rows_valid': 724, 'data_total_rows_test': 4344, 'model_param_count': 10750213, 'fit_dt_train_start': '2023-04-02 23:49:55', 'fit_dt_train_end': '2023-04-02 23:50:22', 'fit_dt_train_duration_seconds': '27', 'fit_stopped_epoch1': 8, 'fit_stopped_epoch2': 6, 'fit_stopped_epoch3': 3, 'fit_num_epochs': 20, 'fit_val_acc': 0.738950252532959, 'fit_train_acc': 0.986187815666198

<Figure size 800x800 with 0 Axes>

<Figure size 800x800 with 0 Axes>